# 간단한 멀티퍼셉트론 구현 (순전파 및 역전파)
다음은 Python의 numpy를 사용하여 간단한 MLP를 구현한 코드입니다.

### 코드 설명
1. **활성화 함수**: Sigmoid 함수와 그 미분 함수를 정의했습니다.
2. **손실 함수**: Mean Squared Error (MSE) 함수로 정의했습니다.
3. **MLP 클래스**:
   - 초기화 함수에서 가중치와 편향을 랜덤으로 초기화합니다.
   - `forward` 메서드에서 순전파를 계산합니다.
   - `backward` 메서드에서 역전파를 계산하여 가중치와 편향을 업데이트합니다.
   - `train` 메서드에서 주어진 에포크 동안 학습을 수행합니다.
4. **데이터**: XOR 문제를 해결하기 위해 입력과 출력을 정의했습니다.
5. **훈련 및 예측**: MLP 모델을 학습시키고, 학습 후 예측을 출력합니다.


In [1]:
import numpy as np

# 활성화 함수 및 그 미분 함수
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

In [2]:
# 손실 함수 (Mean Squared Error)
def mse_loss(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

In [3]:
# MLP 클래스 정의
class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        # 가중치 초기화
        self.weights_input_hidden = np.random.rand(input_size, hidden_size)
        self.bias_hidden = np.zeros((1, hidden_size))
        self.weights_hidden_output = np.random.rand(hidden_size, output_size)
        self.bias_output = np.zeros((1, output_size))

    def forward(self, X):
        # 순전파 계산
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = sigmoid(self.hidden_input)
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = sigmoid(self.final_input)
        return self.final_output

    def backward(self, X, y, learning_rate):
        # 손실의 미분 계산 (역전파)
        output_error = y - self.final_output
        output_delta = output_error * sigmoid_derivative(self.final_output)

        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.hidden_output)

        # 가중치 및 편향 업데이트
        self.weights_hidden_output += self.hidden_output.T.dot(output_delta) * learning_rate
        self.bias_output += np.sum(output_delta, axis=0, keepdims=True) * learning_rate
        self.weights_input_hidden += X.T.dot(hidden_delta) * learning_rate
        self.bias_hidden += np.sum(hidden_delta, axis=0, keepdims=True) * learning_rate

    def train(self, X, y, learning_rate, epochs):
        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y, learning_rate)
            if epoch % 1000 == 0:
                loss = mse_loss(y, self.final_output)
                print(f"Epoch {epoch}, Loss: {loss}")

In [4]:
# 데이터 예제 (XOR 문제)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

# MLP 모델 초기화 및 학습
mlp = MLP(input_size=2, hidden_size=2, output_size=1)
mlp.train(X, y, learning_rate=0.1, epochs=100) # epochs=100000

# 예측
print("Predictions after training:")
print(mlp.forward(X))

Epoch 0, Loss: 0.26490089531768896
Predictions after training:
[[0.49584349]
 [0.50723671]
 [0.50085923]
 [0.5122533 ]]


# 단층 퍼셉트론

# 단층 퍼셉트론의 한계와 XOR 게이트 문제

1. **단층 퍼셉트론의 구현 가능 범위**:
   - 단층 퍼셉트론은 **AND, NAND, OR 게이트** 구현 가능.
   - **XOR 게이트는 구현 불가**.
   
2. **이유**:
   - 단층 퍼셉트론은 **직선 하나로 두 영역을 분리**할 수 있는 문제만 해결 가능.
   - XOR 게이트의 출력값 분포는 직선 하나로 나눌 수 없는 구조를 가짐.

3. **시각적 이해**:
   - **AND, NAND, OR 게이트**는 하얀색 원(출력 0)과 검은색 원(출력 1)을 직선으로 구분 가능.
   - **XOR 게이트**는 하얀색 원과 검은색 원을 **단일 직선으로 분리 불가**; 최소 두 개의 선이 필요.

4. **해결 방법: 다층 퍼셉트론**:
   - 다층 퍼셉트론은 **여러 층의 뉴런을 사용해 비선형 분류** 가능.
   - XOR 게이트와 같은 문제도 다층 퍼셉트론으로 해결 가능.

In [5]:
## AND
def AND(x1, x2):
    w1,w2, theta = 0.5, 0.5, 0.7 #theta
    tmp = x1*w1 + x2*w2
    if tmp <= theta:
        return 0
    elif tmp > theta:
        return 1

In [6]:
AND(1,1)

1

In [7]:
## NOT AND
def NAND(x1, x2):
    w1,w2, theta = -0.5, -0.5, -0.7
    tmp = x1*w1 + x2*w2
    if tmp <= theta:
        return 0
    elif tmp > theta:
        return 1

In [8]:
NAND(1,0)

1

In [ ]:
## OR
def OR(x1, x2):
    x = np.array([x1, x2])
    w = np.array([0.5, 0.5]) # AND와는 가중치(w와 b)만 다르다.
    b = -0.2
    tmp = np.sum(w*x) + b
    if tmp <= 0:
        return 0
    else:
        return 1

In [10]:
OR(1,1)

1

In [11]:
## NOR 
def NOR(x1, x2):
    x = np.array([x1, x2])
    w = np.array([0.5, 0.5]) # AND와는 가중치(w와 b)만 다르다.
    b = -0.2
    tmp = np.sum(w*x) + b
    if tmp <= 0:
        return 0
    else:
        return 1

In [13]:
## XOR
## - XOR문제가 해결이 안됨!
## - 단층 퍼셉트론으로 해결 안되는 문제를 두층을 쌓아서 해결

def XOR(x1,x2):
    s1 = NAND(x1,x2)
    s2 = OR(x1,x2)
    y = AND(s1,s2)
    return y